<a href="https://colab.research.google.com/github/xuan1905/misc/blob/main/optillm_cot_decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --no-cache-dir -U git+https://github.com/codelion/optillm.git

  Cloning https://github.com/codelion/optillm.git to /tmp/pip-req-build-aldi241f
  Running command git clone --filter=blob:none --quiet https://github.com/codelion/optillm.git /tmp/pip-req-build-aldi241f
  Resolved https://github.com/codelion/optillm.git to commit d99727a3771e5f920a7ce70274ae78f260efe3ba
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 kB 167.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 142.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 136.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 135.5 MB/s eta 0:00:00
   ━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optillm.cot_decoding import cot_decode
import torch

def get_device():
  if torch.cuda.is_available():
    return torch.device("cuda")
  else:
    return torch.device("cpu")

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

messages = [
    {"role": "user", "content": "In a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enrolled in jazz dance, and the rest enrolled in hip-hop dance. What percentage of the entire students enrolled in hip-hop dance?"}
]

# Generate the response using CoT decoding
result, confidence = cot_decode(model, tokenizer, messages, aggregate_paths=True, max_new_tokens=512)
device = get_device()
model = model.to(device)
# print(f"Using device: {get_device()}")

print(f"CoT Decoding:\n {result}")
# print(f"Confidence: {confidence}")

input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
input_ids = input_ids.to(device)
input_length = input_ids.shape[1]
output_ids = model.generate(
    input_ids,
    max_new_tokens=512,  # Adjust this value based on how long you want the response to be
    do_sample=False,     # This ensures greedy decoding
    num_beams=1,
)
output_ids = output_ids.cpu()
# Decode only the newly generated tokens
response = tokenizer.decode(output_ids[0][input_length:], skip_special_tokens=True)

print(f"Greedy Decoding:\n {response}")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:589: UserWarning: `do_sample` is set to `False`. Howeve

CoT Decoding:
 To determine the percentage of students enrolled in hip-hop dance, we will follow these steps:

1. **Calculate the number of students enrolled in contemporary dance:**
   \[
   \text{Number of students in contemporary dance} = 20\% \times 20 = 20 \times 0.20 = 4
   \]

2. **Calculate the number of students remaining after the enrollment in contemporary dance:**
   \[
   \text{Remaining students} = 20 - 4 = 16
   \]

3. **Calculate the number of students enrolled in jazz dance:**
   \[
   \text{Number of students in jazz dance} = 25\% \times 16 = 0.25 \times 16 = 4
   \]

4. **Calculate the number of students remaining after the enrollment in jazz dance:**
   \[
   \text{Remaining students} = 16 - 4 = 12
   \]

5. **Calculate the number of students enrolled in hip-hop dance by dividing the remaining students by the total number of students and multiplying by 100 to convert to a percentage:**
   \[
   \text{Number of students in hip-hop dance} = \left( \frac{12}{20} \right